In [ ]:
%matplotlib inline

# Quantile estimation using a Gaussian process regressor.


In [ ]:
from __future__ import annotations

from gemseo import configure
from gemseo import configure_logger
from gemseo import sample_disciplines
from gemseo.algos.design_space import DesignSpace
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.disciplines.analytic import AnalyticDiscipline
from gemseo.disciplines.surrogate import SurrogateDiscipline
from gemseo.mlearning.regression.algos.gpr import GaussianProcessRegressor
from gemseo.uncertainty.statistics.empirical_statistics import EmpiricalStatistics

from gemseo_mlearning.active_learning.active_learning_algo import ActiveLearningAlgo
from gemseo_mlearning.active_learning.visualization.acquisition_view import (
    AcquisitionView,
)

configure(False, False, True, False, False, False, False)
configure_logger()

We consider the Rosenbrock function $f(x,y)=(1-x)^2+100(y-x^2)^2$:



In [ ]:
discipline = AnalyticDiscipline({"z": "(1-x)**2+100*(y-x**2)**2"}, name="Rosenbrock")

defined over the input space $[-2,2]^2$:



In [ ]:
input_space = DesignSpace()
input_space.add_variable("x", l_b=-2, u_b=2, value=1.0)
input_space.add_variable("y", l_b=-2, u_b=2, value=1.0)

with the uncertain space:



In [ ]:
uncertain_space = ParameterSpace()
uncertain_space.add_random_variable("x", "OTUniformDistribution", minimum=-2, maximum=2)
uncertain_space.add_random_variable("y", "OTUniformDistribution", minimum=-2, maximum=2)

First,
we create an initial training dataset using an optimal LHS including 30 samples:



In [ ]:
learning_dataset = sample_disciplines([discipline], input_space, "z", 30, "OT_OPT_LHS")

and an initial Gaussian process regressor:



In [ ]:
gpr = GaussianProcessRegressor(learning_dataset)

Then,
we look for 20 points that will help us get closer to the minimum;
by default,
for this purpose,
the active learning algorithm looks for the point maximizing the U-function.



In [ ]:
active_learning = ActiveLearningAlgo(
    "Quantile",
    input_space,
    gpr,
    level=0.8,
    uncertain_space=uncertain_space,
    # criterion_name="EF",
)
# active_learning.set_acquisition_algorithm("fullfact", n_samples=30 * 30)
# active_learning.set_acquisition_algorithm("NELDER-MEAD")
# active_learning.set_acquisition_algorithm("SLSQP")
active_learning.set_acquisition_algorithm("DIFFERENTIAL_EVOLUTION")
active_learning.acquire_new_points(discipline, 20, show=True)

Lastly,
we plot the training points,
the original model,
the Gaussian process regressor
and the U-function
after the last acquisition:



In [ ]:
acquisition_view = AcquisitionView(active_learning)
acquisition_view.draw(criterion_title="U-function", discipline=discipline)

dataset = sample_disciplines([discipline], uncertain_space, "z", 10000, "OT_OPT_LHS")
reference_quantile = EmpiricalStatistics(dataset, ["z"]).compute_quantile(0.8)
dataset = sample_disciplines(
    [SurrogateDiscipline(gpr)], uncertain_space, "z", 10000, "OT_OPT_LHS"
)
estimated_quantile = EmpiricalStatistics(dataset, ["z"]).compute_quantile(0.8)
print(reference_quantile, estimated_quantile)  # noqa